## 6.6 逻辑回归参数调优

建立 LogisticRegression 对象时，可以设置很多参数，设置正则化时的参数 `penalty` 和 `C`，设置类型权重的参数 `class_weight` 最为常用。下面我们在逻辑回归中进行调参，看能否优化模型效果。

过度拟合是指模型复杂度过高，导致所选模型对已知数据（训练集）预测得很好，但对未知数据（测试集）预测很差。


![](https://ai-studio-static-online.cdn.bcebos.com/9d08336320d54a88a6caeb616a96f83873493388d076408d9dc8eda17cbc4e1f)


关于模型拟合的直观展示如下：
![](https://ai-studio-static-online.cdn.bcebos.com/e40d57e432bd461f8d97419a907cf665348c12c37e1742ac8fe3cb3b7f52ff5b)


所谓正则化，就是通过在模型中添加一些惩罚项或约束条件来控制模型复杂度，达到减轻过度拟合的目的。正则化常用的为 $L_1$ 和 $L_2$ 正则化，其中存在正则化系数 $\lambda (\lambda>0)$ 为依赖于 `C` 的常数，正则化系数 $\lambda$ 越大则惩罚力度越大。
- `penalty` -- 可设为 `l1` 或 `l2` ，分别代表 $L_1$ 和 $L_2$ 正则化，默认为 `l2`。
- ` C` --  `C` 为正则化系数 $\lambda$  的倒数，必须为正数，默认为 1。值越小，代表正则化越强。
- `class_weight` -- 默认为 `None`，可设置为 `balanced`，即根据训练样本量来分配权重。某种类型样本量越多，则权重越低，样本量越少，则权重越高。


## 实训任务

- 建立一个 LogisticRegression 对象，命名为 `lr`，要求使用 `l1` 正则化，参数 `C` 设置为 `0.6`，参数 `class_weight` 设置为 `balanced`。
- 对 `lr` 对象调用 `fit` 方法，带入训练集 `x_train, y_train` 进行训练
- 对训练好的 `lr` 模型调用 `predict_proba` 方法,带入测试集 `x_test` 进行预测，将结果保存到变量 `y_predict` 中
- 调用 `roc_auc_score` 方法，将 `y_test，y_predict` 作为输入参数，求出测试集准确率值，将结果赋予 `test_auc`。


可以看到，对模型进行三种参数的调整后，准确率有了些许的提升。除了在模型参数上进行调整外，还可以在数据集本身进行一些优化，标准化和离散化是两种比较常用的技巧。 

In [1]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_table('dataset11(不良记录).txt', sep='\t')
y = data['Default'].values
x = data.drop(['Default'], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=33, stratify=y)

# 建立一个LogisticRegression对象，命名为lr
lr = LogisticRegression(penalty='l1', C=0.6, class_weight='balanced', solver='liblinear')

# 对lr对象调用fit方法，带入训练集x_train, y_train进行训练
lr.fit(x_train, y_train)

# 对训练好的lr模型调用predict_proba方法
y_predict = lr.predict_proba(x_test)[:, 1]

# 调用roc_auc_score方法
test_auc = roc_auc_score(y_test, y_predict)

print('逻辑回归模型test auc:')
print(test_auc)

逻辑回归模型test auc:
0.8776952288366846


#### 问题：上述三种参数的调整对于模型性能提升的贡献分别是多少呢？请在下方进行消融实验 （ablation study）验证。

In [13]:

#消融实验:它通过逐步去除或禁用模型的特定部分，观察性能的变化，以判断每个部分对整体模型性能的贡献

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

data = pd.read_table('dataset11(不良记录).txt', sep='\t')
y = data['Default'].values
x = data.drop(['Default'], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=33, stratify=y)

# 使用默认参数训练模型
lr_default = LogisticRegression()
lr_default.fit(x_train, y_train)
y_predict_default = lr_default.predict_proba(x_test)[:, 1]
auc_default = roc_auc_score(y_test, y_predict_default)

# 仅调整 penalty 参数为 l1
lr_penalty = LogisticRegression(penalty='l1', solver='liblinear')
lr_penalty.fit(x_train, y_train)
y_predict_penalty = lr_penalty.predict_proba(x_test)[:, 1]
auc_penalty = roc_auc_score(y_test, y_predict_penalty)

# 仅调整 C 参数为 0.6
lr_C = LogisticRegression(C=0.6)
lr_C.fit(x_train, y_train)
y_predict_C = lr_C.predict_proba(x_test)[:, 1]
auc_C = roc_auc_score(y_test, y_predict_C)

# 仅调整 class_weight 参数为 balanced
lr_class_weight = LogisticRegression(class_weight='balanced')
lr_class_weight.fit(x_train, y_train)
y_predict_class_weight = lr_class_weight.predict_proba(x_test)[:, 1]
auc_class_weight = roc_auc_score(y_test, y_predict_class_weight)

# 使用所有调整后的参数训练模型
lr_all = LogisticRegression(penalty='l1', C=0.6, class_weight='balanced', solver='liblinear')
lr_all.fit(x_train, y_train)
y_predict_all = lr_all.predict_proba(x_test)[:, 1]
auc_all = roc_auc_score(y_test, y_predict_all)

# 计算提升百分比
def calculate_improvement(default_auc, adjusted_auc):
    return (adjusted_auc - default_auc) / default_auc * 100

improvement_penalty = calculate_improvement(auc_default, auc_penalty)
improvement_C = calculate_improvement(auc_default, auc_C)
improvement_class_weight = calculate_improvement(auc_default, auc_class_weight)
improvement_all = calculate_improvement(auc_default, auc_all)

print('默认参数模型 AUC:', auc_default)
print('仅调整 penalty 参数模型 AUC:', auc_penalty, '提升百分比:', improvement_penalty)
print('仅调整 C 参数模型 AUC:', auc_C, '提升百分比:', improvement_C)
print('仅调整 class_weight 参数模型 AUC:', auc_class_weight, '提升百分比:', improvement_class_weight)
print('所有参数调整后的模型 AUC:', auc_all, '提升百分比:', improvement_all)

默认参数模型 AUC: 0.6493349350319194
仅调整 penalty 参数模型 AUC: 0.872667553710911 提升百分比: 34.394055614459305
仅调整 C 参数模型 AUC: 0.6468228704664456 提升百分比: -0.38686730529142543
仅调整 class_weight 参数模型 AUC: 0.7792301808893791 提升百分比: 20.00435196838353
所有参数调整后的模型 AUC: 0.8777189597251362 提升百分比: 35.171990966725076


从消融实验可以得出：调整 penalty 参数模型提升最为明显，但是同时调整三个参数提升比率和仅仅调整oenalty参数模型相差不大，因此继续进行实验.

In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_table('dataset11(不良记录).txt', sep='\t')
y = data['Default'].values
x = data.drop(['Default'], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=33, stratify=y)

# 使用默认参数训练模型
lr_default = LogisticRegression()
lr_default.fit(x_train, y_train)
y_predict_default = lr_default.predict_proba(x_test)[:, 1]
auc_default = roc_auc_score(y_test, y_predict_default)

# 使用所有调整后的参数训练模型
lr_all = LogisticRegression(penalty='l1', C=0.6, class_weight='balanced', solver='liblinear')
lr_all.fit(x_train, y_train)
y_predict_all = lr_all.predict_proba(x_test)[:, 1]
auc_all = roc_auc_score(y_test, y_predict_all)

# 不调整 penalty 参数
lr_no_penalty = LogisticRegression(C=0.6, class_weight='balanced', solver='liblinear')
lr_no_penalty.fit(x_train, y_train)
y_predict_no_penalty = lr_no_penalty.predict_proba(x_test)[:, 1]
auc_no_penalty = roc_auc_score(y_test, y_predict_no_penalty)

# 不调整 C 参数
lr_no_C = LogisticRegression(penalty='l1', class_weight='balanced', solver='liblinear')
lr_no_C.fit(x_train, y_train)
y_predict_no_C = lr_no_C.predict_proba(x_test)[:, 1]
auc_no_C = roc_auc_score(y_test, y_predict_no_C)

# 不调整 class_weight 参数
lr_no_class_weight = LogisticRegression(penalty='l1', C=0.6, solver='liblinear')
lr_no_class_weight.fit(x_train, y_train)
y_predict_no_class_weight = lr_no_class_weight.predict_proba(x_test)[:, 1]
auc_no_class_weight = roc_auc_score(y_test, y_predict_no_class_weight)

# 计算提升百分比
def calculate_improvement(default_auc, adjusted_auc):
    return (adjusted_auc - default_auc) / default_auc * 100

improvement_all = calculate_improvement(auc_default, auc_all)
improvement_no_penalty = calculate_improvement(auc_default, auc_no_penalty)
improvement_no_C = calculate_improvement(auc_default, auc_no_C)
improvement_no_class_weight = calculate_improvement(auc_default, auc_no_class_weight)

print('默认参数模型 AUC:', auc_default)
print('所有参数调整后的模型 AUC:', auc_all, '提升百分比:', improvement_all)
print('仅不调整 penalty 参数模型 AUC:', auc_no_penalty, '提升百分比:', improvement_no_penalty)
print('仅不调整 C 参数模型 AUC:', auc_no_C, '提升百分比:', improvement_no_C)
print('仅不调整 class_weight 参数模型 AUC:', auc_no_class_weight, '提升百分比:', improvement_no_class_weight)

默认参数模型 AUC: 0.6493349350319194
所有参数调整后的模型 AUC: 0.8777065487432449 提升百分比: 35.17007962925935
仅不调整 penalty 参数模型 AUC: 0.8633726829651499 提升百分比: 32.96261087857671
仅不调整 C 参数模型 AUC: 0.8776728617923968 提升百分比: 35.1648917132809
仅不调整 class_weight 参数模型 AUC: 0.8727084690358277 提升百分比: 34.400356726983716


虽然仅调整C参数模型之后AUC有些微下降，但是如果仅不调整C参数，模型AUC仍有所下降？没能搞明白？感觉和我理解的正则化有点冲突？


## 6.7 使用标准化提升逻辑回归模型效果

部分数据，特别是与金额相关的数据，其取值跨度较大且值相对分散，以最近 12 个月取现金额均值 cashAmt_mean 为例，其统计信息为：

![](https://ai-studio-static-online.cdn.bcebos.com/8cdadbd06a274dcab6fe57554fca7c2f667e830b0f7f455f9b8de5bdcc7bed02)


其平均值为 1471，最大值为 72633。标准差较大为 2892，表明数据较分散，这对于模型预测效果会有影响，为此我们需要对数据进行标准化处理。数据的标准化（normalization）是将全部数据平移到原点，实现数据的均值中心化，进一步按比例缩放，实现标准差单位化，使得数据不同变量维度的量纲一致，数据的分布更加规范化，为模型学习提供较规整的数据基础，提高优化过程的健壮性，比如，基于梯度下降的优化中能够快速收敛到最优值，降低不规范数据的干扰。


在机器学习中对特征做归一化目的有:

1,避免训练得到的模型权重过小,引起数值计算不稳定;

2,使参数优化时能以较快的速度收敛.



较常用的有 Z-score 标准化：

Z-score 标准化，数据处理后符合标准正态分布，即均值为 0，标准差为 1 

![](https://ai-studio-static-online.cdn.bcebos.com/319787195e1542da8f35627f8bdd85ae3975dd6667ab40599114372f9c40a16c)



## 实训任务

对 data 中所有连续型的列 `continuous_columns` 进行 Z-score 标准化（直接在原始数据 data 上进行标准化）

可以看到，标准化后的数据，带入在之前已做过正则化的模型，相较于未标准化的情况，其 AUC 值有显著提升。

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
data = pd.read_table('dataset11(不良记录).txt',sep='\t')
continuous_columns = ['age','cashTotalAmt','cashTotalCnt','monthCardLargeAmt','onlineTransAmt','onlineTransCnt','publicPayAmt','publicPayCnt','transTotalAmt','transTotalCnt','transCnt_non_null_months','transAmt_mean','transAmt_non_null_months','cashCnt_mean','cashCnt_non_null_months','cashAmt_mean','cashAmt_non_null_months','card_age', 'trans_total','total_withdraw', 'avg_per_withdraw','avg_per_online_spend', 'avg_per_public_spend', 'bad_record']

# 对data中所有连续型的列进行Z-score标准化

# Create a StandardScaler object
scaler = StandardScaler()

# Standardize the continuous columns in the data
data[continuous_columns] = scaler.fit_transform(data[continuous_columns])

# 查看标准化后的数据的均值和标准差，以cashAmt_mean为例
print('cashAmt_mean标准化后的均值：',data['cashAmt_mean'].mean())
print('cashAmt_mean标准化后的标准差：',data['cashAmt_mean'].std())

# 查看标准化后对模型的效果提升
y = data['Default'].values
x = data.drop(['Default'], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state = 33,stratify=y)



lr = LogisticRegression(penalty='l2',C=0.6,class_weight='balanced')
lr.fit(x_train, y_train)

# 查看模型预测结果
y_predict = lr.predict_proba(x_test)[:,1]
auc_score =roc_auc_score(y_test,y_predict)
print('score:',auc_score)


cashAmt_mean标准化后的均值： 8.406657906398548e-18
cashAmt_mean标准化后的标准差： 1.00001056384524
score: 0.8773915007413857


## 6.8 使用离散化提升逻辑回归模型效果

在处理连续型数据时，为了便于分析需要将其离散化，即把数据放入一个个小区间中。若每个区间的间隔是相等的，这类离散化称作等距离散化。另外一种离散化叫做等频离散化，它是指每个区间的样本数相同。

Pandas 提供了 `qcut()` 函数可实现等频离散化，自动分配每个面元的区间，其函数语法为：`qcut(x, q, duplicates='raise'...)`，其中：
- `x` --  1 维 ndarray 或 Series，表示要划分的数据。
- `q` -- 分位数，用来指定区间的数量，表示要划分为几组。
- `duplicates` -- 如果分组的 bin 边缘不唯一，`duplicates` 为`‘raise’则`报告 ValueError，为`‘drop’`则直接删除非唯一值。



## 实训任务

对data中所有数值连续型的列 `continuous_columns` 利用 `qcut` 进行等频离散化，将每一列都离散为5组，设置 `duplicates` 为 `drop` 以删除非唯一值（直接在原始数据 data 中进行离散化）。

可以看到，离散化后的数据，带入原始模型中，相较于未离散化的情况，其AUC值也有了明显提升。

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_table('dataset12.txt',sep='\t')
continuous_columns = ['age','cashTotalAmt','cashTotalCnt','monthCardLargeAmt','onlineTransAmt','onlineTransCnt','publicPayAmt','publicPayCnt','transTotalAmt','transTotalCnt','transCnt_non_null_months','transAmt_mean','transAmt_non_null_months','cashCnt_mean','cashCnt_non_null_months','cashAmt_mean','cashAmt_non_null_months','card_age', 'trans_total', 'total_withdraw', 'avg_per_withdraw','avg_per_online_spend', 'avg_per_public_spend', 'bad_record']

# 对data中数值连续型的列进行等频离散化，将每一列都离散为5个组。
for col in continuous_columns:
    data[col] = pd.qcut(data[col], q=5, duplicates='drop')


# 查看离散化后的数据
print(data.head())

# 查看离散化后对模型的效果提升
# 先对各离散组进行One-Hot处理
data=pd.get_dummies(data)
y = data['Default'].values
x = data.drop(['Default'], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state = 33,stratify=y)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

lr = LogisticRegression(penalty='l2',C=0.6,class_weight='balanced')
lr.fit(x_train, y_train)

# 查看模型预测结果
y_predict = lr.predict_proba(x_test)[:,1]
score_auc = roc_auc_score(y_test,y_predict)
print('score:',score_auc)

                age                       card_age     cashAmt_mean  \
0    (0.176, 0.909]  (-1.4649999999999999, -0.876]  (-0.51, -0.457]   
1    (0.176, 0.909]              (-0.406, 0.00474]  (-0.51, -0.457]   
2    (0.176, 0.909]               (-0.876, -0.406]  (-0.51, -0.457]   
3    (0.176, 0.909]               (-0.876, -0.406]  (0.286, 24.604]   
4  (-2.021, -0.922]  (-1.4649999999999999, -0.876]  (-0.51, -0.457]   

  cashAmt_non_null_months      cashCnt_mean cashCnt_non_null_months  \
0        (-0.815, -0.429]  (-0.872, -0.459]        (-0.817, -0.432]   
1        (-0.815, -0.429]  (-0.872, -0.459]        (-0.817, -0.432]   
2        (-0.815, -0.429]  (-0.872, -0.459]        (-0.817, -0.432]   
3        (-0.815, -0.429]   (0.594, 20.627]        (-0.817, -0.432]   
4        (-0.815, -0.429]  (-0.872, -0.459]        (-0.817, -0.432]   

       cashTotalAmt      cashTotalCnt  inCourt  isBlackList  ...  sex_2  \
0   (-0.34, -0.259]  (-0.535, -0.431]        0            0  ...      1

#### 总结：各种方案的性能比较 （注意比较的公平性）

#### 请在此处作答：1.模型参数调优后score：0.8776952288366846；2.使用标准化提升逻辑回归模型效果score: 0.8773915007413857;3.使用离散化提升逻辑回归模型效果score: 0.910711509317238
三种方案提升逻辑回归模型效果对比可得，单独采用离散化提升效果最好。


但是， 模型参数调优是指选择最佳的超参数组合，以获得最优的模型性能；特征标准化是将连续特征进行缩放，使其具有零均值和单位方差；特征离散化是将连续特征划分为不同的区间或类别。


显然，标准化和离散化其实处理的是不同方面的特征，因此我尝试了同时采用以上三种方式来提升模型效果。


另外，性能比较不仅在于采用何种方式提升最高，例如采用标准化提升逻辑回归模型效果，和模型参数调优提升效果相近，但是模型参数调优仅需要调整超参数，占用运算和存储资源可能更少。

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import KBinsDiscretizer
import pandas as pd

# 读取数据
data = pd.read_table('dataset11(不良记录).txt', sep='\t')
y = data['Default'].values
x = data.drop(['Default'], axis=1).values

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=33, stratify=y)

# 离散化
discretizer = KBinsDiscretizer(n_bins=10, encode='onehot', strategy='uniform')
x_train_binned = discretizer.fit_transform(x_train)
x_test_binned = discretizer.transform(x_test)

# 模型参数调优
param_grid = {'C': [0.1, 0.6, 1, 10], 'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None]}
lr = LogisticRegression(solver='liblinear')
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(x_train_binned, y_train)

# 最佳模型
best_lr = grid_search.best_estimator_
y_predict = best_lr.predict_proba(x_test_binned)[:, 1]

# 计算AUC
from sklearn.metrics import roc_auc_score
test_auc = roc_auc_score(y_test, y_predict)
print('最佳模型 AUC:', test_auc)

最佳模型 AUC: 0.8876435173213665


In [ ]:
然而，这个模型的AUC值竟然比之前的模型还要低。标准化将数据缩放到相同的范围，但离散化会将连续特征转换为离散特征，这可能会导致信息丢失。特别是当离散化的分箱数较少时，可能会丢失特征的细节信息。此外，同时进行标准化和离散化可能会导致特征冗余。例如，标准化后的特征和离散化后的特征可能包含重复的信息，这会增加模型的复杂性，导致过拟合或欠拟合。


In [18]:
#选择了另外的方法，仅进行标准化和参数调优。
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 读取数据
data = pd.read_table('dataset11(不良记录).txt', sep='\t')
y = data['Default'].values
x = data.drop(['Default'], axis=1).values

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=33, stratify=y)

# 标准化
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# 模型参数调优
param_grid = {'C': [0.1, 0.6, 1, 10], 'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None]}
lr = LogisticRegression(solver='liblinear')
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(x_train_scaled, y_train)

# 最佳模型
best_lr = grid_search.best_estimator_
y_predict = best_lr.predict_proba(x_test_scaled)[:, 1]

# 计算AUC
from sklearn.metrics import roc_auc_score
test_auc = roc_auc_score(y_test, y_predict)
print('最佳模型 AUC:', test_auc)


最佳模型 AUC: 0.8776330375428112


In [19]:
#仅进行离散化和超参数调优
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import KBinsDiscretizer
import pandas as pd

# 读取数据
data = pd.read_table('dataset11(不良记录).txt', sep='\t')
y = data['Default'].values
x = data.drop(['Default'], axis=1).values

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=33, stratify=y)

# 离散化
discretizer = KBinsDiscretizer(n_bins=10, encode='onehot', strategy='uniform')
x_train_binned = discretizer.fit_transform(x_train)
x_test_binned = discretizer.transform(x_test)

# 模型参数调优
param_grid = {'C': [0.1, 0.6, 1, 10], 'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None]}
lr = LogisticRegression(solver='liblinear')
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(x_train_binned, y_train)

# 最佳模型
best_lr = grid_search.best_estimator_
y_predict = best_lr.predict_proba(x_test_binned)[:, 1]

# 计算AUC
from sklearn.metrics import roc_auc_score
test_auc = roc_auc_score(y_test, y_predict)
print('最佳模型 AUC:', test_auc)

最佳模型 AUC: 0.8876406532486224
